# Extraction and Processing of Data from Records of Euler's Pendulum
Bottom Up Orthographic Perspective;
Camera concentric to rod at equilibrium

## Frame Decomposition

### Variables: Put the names here

In [ ]:
video_path = '' # Put the full name of the recording to be processed. make sure it is under the same folder in which you are
output_directory = '' # Name of the folder to store frame, make VERY sure it is UNDER the same follder in which you are in

### Decomposition
 - Run It.

In [ ]:
import cv2, os

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

video_capture = cv2.VideoCapture(video_path)

frame_count = 0

while True:
    success, frame = video_capture.read()
    if not success:
        break

    output_path = os.path.join(output_directory, f"frame_{frame_count}.jpg")
    cv2.imwrite(output_path, frame)
    frame_count += 1

video_capture.release()
print(f"Total frames extracted: {frame_count}")


## Check if the mask is applied correctly
 - Check the other code I shared.
 - run it
 - edit the upper_color and lower_color values respectively, if their persists an issue with it. run before editing it.

In [ ]:
import numpy as np

upper_color = np.array([179, 255, 255]) # Upper limit of HSV
lower_color = np.array([28, 6, 122]) # Lower Limit of HSV

### Check the Mask... check for a whitish portion visible corresponding to the white section of the rod

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageFilter, Image
img = Image.open(output_directory + '/' + 'frame_1000' )
cpr = img.crop(
    (200, 500, 900, 1400)
)

ftd = cpr.filter(ImageFilter.GaussianBlur(radius=0))

ftd.save('Chopped.jpg')
#image = cv2.imread(cpr)
image = cv2.imread('Chopped.jpg')

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

mask = cv2.inRange(hsv, lower_color, upper_color)

# Step 5: Apply the mask to extract the region of interest
segmented_image = cv2.bitwise_and(image, image, mask=mask)

# Show the segmented image
fig = plt.figure()
fig.add_subplot()
plt.imshow(mask)
plt.colorbar()

## Check if the white section is detected

In [ ]:
from math import sqrt

edges = cv2.Canny(mask, 0, 300)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=3, minLineLength=10, maxLineGap=50)

highest_line = None
highest = 0
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if sqrt((x2-x1)**2 + (y2-y1)**2) > highest:
            highest_line = line
            highest = sqrt((x2-x1)**2 + (y2-y1)**2)
else:
    highest = 0


# Draw the highest line on the image
if highest_line is not None:
    x1, y1, x2, y2 = highest_line[0]
    cv2.line(hsv, (x1, y1), (x2, y2), (0, 255, 0), 2)
plt.imshow(hsv)
print(highest)


## Track the data - Proceed if everything above works as expected

In [ ]:
import cv2
import numpy as np

from PIL import ImageFilter

output_directory = 'Cap_Frames_01'
def track(frame_number):    
    img = Image.open(output_directory + '/frame_' + str(frame_number) + '.jpg')
    cpr = img.crop(
        (200, 500, 900, 1400)
    )

    ftd = cpr.filter(ImageFilter.GaussianBlur(radius=0))

    ftd.save('Chopped.jpg')
    #image = cv2.imread(cpr)
    image = cv2.imread('Chopped.jpg')

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, lower_color, upper_color)

    edges = cv2.Canny(mask, 0, 300)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=3, minLineLength=10, maxLineGap=50)

    highest_line = None
    highest = 0
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            if sqrt((x2-x1)**2 + (y2-y1)**2) > highest:
                highest_line = line
                highest = sqrt((x2-x1)**2 + (y2-y1)**2)
    else:
        highest = 0

    if highest_line is not None:
        x1, y1, x2, y2 = highest_line[0]
        cv2.line(hsv, (x1, y1), (x2, y2), (0, 255, 0), 2)

    return sqrt((330 - x2)**2 + (420 - y2)**2)

### Obtain the Displacement

In [ ]:
initial_frame = 1 # Enter the first frame number in the directory

In [ ]:
Deflection_data = np.array([])
total_frames = frame_count

for i in range(initial_frame, total_frames):
    Deflection_data = np.append(Deflection_data, track(i))

### Obtain $ \theta $ from `Deflection_data`

In [ ]:
from math import asin

rod_length = None
data_01 = np.array([])

for i in range(0, len(Deflection_data)):
    data_01 = np.append(data_01, asin(Deflection_data[i]/rod_length))

### Plot and See...

In [ ]:
import matplotlib.pyplot as plt

First = None
Last = None

plt.scatter(range(First, Last), data_01[First: Last])
plt.title('Deflection Theta W.R.T. Frame')
plt.xlabel('Frame Number')
plt.ylabel('Deflection (Radians)')

## Save the obtained data

In [ ]:
file_name = '' # Desired name of the text file in which the data will be put. Create a separate file for each recording you process

with open(file_path, 'w') as file:
    for i in range(0, len(data_01)):
        file.write(str(data_01[i]) + ', ' + '\n')
close(file_name)